In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
host = 'https://api.census.gov/data'
year = '/2021'
dataset_acronym = '/acs/acs1'
g = '?get='
variables = 'NAME,B19083_001E'
location = '&for=state:*&key='
# I have my API Key saved in a Class in a separate file so my
# information can't be seen :)
usr_key = "0d20bc54b2f22dccdd48690f3ef9c523a9b7da70"
#Put it all together in one f-string:
query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
# Use requests package to call out to the API
response = requests.get(query_url)
# Convert the Response to text and print the result
contents = response.text

In [3]:
contents

'[["NAME","B19083_001E","state"],\n["Alabama","0.4823","01"],\n["Puerto Rico","0.5423","72"],\n["Arizona","0.4629","04"],\n["Arkansas","0.4751","05"],\n["California","0.4924","06"],\n["Colorado","0.4604","08"],\n["Connecticut","0.4985","09"],\n["Delaware","0.4547","10"],\n["District of Columbia","0.5305","11"],\n["Florida","0.4869","12"],\n["Georgia","0.4787","13"],\n["Hawaii","0.4552","15"],\n["Idaho","0.4577","16"],\n["Illinois","0.4824","17"],\n["Indiana","0.4480","18"],\n["Iowa","0.4489","19"],\n["Kansas","0.4599","20"],\n["Kentucky","0.4757","21"],\n["Louisiana","0.4988","22"],\n["Maine","0.4663","23"],\n["Maryland","0.4631","24"],\n["Massachusetts","0.4893","25"],\n["Michigan","0.4659","26"],\n["Minnesota","0.4493","27"],\n["Mississippi","0.4808","28"],\n["Missouri","0.4706","29"],\n["Montana","0.4693","30"],\n["Nebraska","0.4560","31"],\n["Nevada","0.4726","32"],\n["New Hampshire","0.4348","33"],\n["New Jersey","0.4853","34"],\n["New Mexico","0.4861","35"],\n["New York","0.5140"

In [4]:
gini = pd.read_json(contents)

In [5]:
gini

,0,1,2
0,NAME,B19083_001E,state
1,Alabama,0.4823,01
2,Puerto Rico,0.5423,72
3,Arizona,0.4629,04
4,Arkansas,0.4751,05
5,California,0.4924,06
6,Colorado,0.4604,08
7,Connecticut,0.4985,09
8,Delaware,0.4547,10
9,District of Columbia,0.5305,11


In [6]:
def create_gini():
    years = ["/2011", "/2012", "/2013", "/2014", "/2015", "/2016", "/2017", "/2018", "/2019", "/2021", 
             "/2022"]
    year = "/2010"
    query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
    response = requests.get(query_url)
    contents = response.text
    gini = pd.read_json(contents)
    year_col = np.full(len(gini), year[1:])
    year_col = pd.DataFrame(year_col)
    gini = pd.concat([gini, year_col], axis = 1)
    for year in years:
        query_url = f"{host}{year}{dataset_acronym}{g}{variables}{location}{usr_key}"
        response = requests.get(query_url)
        contents = response.text
        temp = pd.read_json(contents)
        temp = temp[1:]
        year_col = np.full(len(temp), year[1:])
        year_col = pd.DataFrame(year_col)
        temp = pd.concat([temp, year_col], axis = 1)
        gini = pd.concat([gini, temp])
    return gini

In [7]:
gini = create_gini()

In [8]:
gini

,0,1,2,0
0,NAME,B19083_001E,state,2010
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
...,...,...,...,...
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022
51,Wyoming,0.4437,56,2022
52,Puerto Rico,0.5477,72,NaN


In [9]:
gini

,0,1,2,0
0,NAME,B19083_001E,state,2010
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
...,...,...,...,...
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022
51,Wyoming,0.4437,56,2022
52,Puerto Rico,0.5477,72,NaN


In [10]:
gini.columns = ["State","gini","state", "year"]

In [12]:
gini = gini[gini["State"] != "NAME"]

In [13]:
gini

,State,gini,state,year
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
5,California,0.471,06,2010
...,...,...,...,...
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022
51,Wyoming,0.4437,56,2022
52,Puerto Rico,0.5477,72,NaN


In [14]:
gini = gini.dropna()


In [15]:
gini

,State,gini,state,year
1,Alabama,0.472,01,2010
2,Alaska,0.422,02,2010
3,Arizona,0.455,04,2010
4,Arkansas,0.458,05,2010
5,California,0.471,06,2010
...,...,...,...,...
47,Virginia,0.4755,51,2022
48,Washington,0.4742,53,2022
49,West Virginia,0.4804,54,2022
50,Wisconsin,0.4510,55,2022


In [16]:
gini = gini.drop(["state"], axis = 1)

In [17]:
gini

,State,gini,year
1,Alabama,0.472,2010
2,Alaska,0.422,2010
3,Arizona,0.455,2010
4,Arkansas,0.458,2010
5,California,0.471,2010
...,...,...,...
47,Virginia,0.4755,2022
48,Washington,0.4742,2022
49,West Virginia,0.4804,2022
50,Wisconsin,0.4510,2022


In [18]:
coeffs = pd.to_numeric(gini["gini"])

In [19]:
gini["gini"] = coeffs

In [ ]:
years = pd.to_numeric(gini["year"])
gini["year"] = year